In [2]:
%pip install -q -U keras-tuner
%pip install -q -U tensorboard-plugin-profile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

## Data Fetching

In [4]:
from utils.data_fetch import read_data
data = read_data()

C:\Users\Vardan\AppData\Local\Temp\ipykernel_11492\1164869249.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  data = read_data()


In [5]:
data

,Spindle Speed,Feed (mm/min),Tool Dia. (mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
0,3000.0,10.0,1.0,49.3753,49.3694,0.0059,3.0,1.076451,0.0480,0.18700
1,3000.0,15.0,1.0,49.3694,49.3682,0.0012,4.5,0.327195,0.0640,0.00200
2,3000.0,20.0,1.0,49.3682,49.3659,0.0023,6.0,0.806629,0.0500,0.14500
3,3250.0,10.0,1.0,49.3659,49.3635,0.0024,3.0,1.007537,0.0565,0.14425
4,3250.0,15.0,1.0,49.3635,49.3617,0.0018,4.5,0.553689,0.0605,0.08075
...,...,...,...,...,...,...,...,...,...,...
103,4750.0,15.0,2.0,47.2419,47.2106,0.0313,15.0,10.826819,0.1850,0.08100
104,4750.0,20.0,2.0,47.2106,47.1822,0.0284,20.0,12.085559,0.1660,0.06300
105,5000.0,10.0,2.0,47.1822,47.1535,0.0287,10.0,6.269954,0.1390,0.07600
106,5000.0,15.0,2.0,47.1535,47.1256,0.0279,15.0,9.038995,0.1390,0.12500


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Spindle Speed   108 non-null    float64
 1   Feed (mm/min)   108 non-null    float64
 2   Tool Dia. (mm)  108 non-null    float64
 3   Wi              108 non-null    float64
 4   Wf              108 non-null    float64
 5   delta Weight    108 non-null    float64
 6   MRR ideal       108 non-null    float64
 7   MRR actual      108 non-null    float64
 8   Delta L         108 non-null    float64
 9   Delta W         108 non-null    float64
dtypes: float64(10)
memory usage: 8.6 KB


## Data Preprocessing

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
train_set, test_set = train_test_split(data, test_size=0.2, random_state=32)

In [8]:
X_train_full = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train_full =pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [9]:
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=32)

In [10]:
X_train = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train=pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [11]:
X_val = val_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_val=pd.concat([val_set["Spindle Speed"], val_set["Feed (mm/min)"]], axis=1)

In [12]:
X_test = test_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_test=pd.concat([test_set["Spindle Speed"], test_set["Feed (mm/min)"]], axis=1)

In [13]:
X_train.shape

(68, 8)

In [14]:
norm_layer = keras.layers.Normalization(input_shape=X_train.shape[1:])

In [15]:
model = keras.models.Sequential([
    norm_layer,
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(2)
])
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)

In [16]:
from pathlib import Path
from time import strftime
def get_run_logdir(root_logdir="my_logs"):
 return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() 

In [17]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))

In [18]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 8)                 17        
 on)                                                             
                                                                 
 batch_normalization (Batch  (None, 8)                 32        
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 500)               4500      
                                                                 
 batch_normalization_1 (Bat  (None, 500)               2000      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 300)               150300    
                                                        

In [20]:
history = model.fit(X_train,y_train, epochs=500, verbose=1, callbacks=[ early_stopping_cb,tensorboard_cb],validation_data=(X_val,y_val))

Epoch 1/500


3/3 [==============================] - 3s 240ms/step - loss: 8148610.0000 - root_mean_squared_error: 2854.5771 - val_loss: 7877668.5000 - val_root_mean_squared_error: 2806.7183
Epoch 2/500
3/3 [==============================] - 0s 84ms/step - loss: 8136829.0000 - root_mean_squared_error: 2852.5129 - val_loss: 7870610.5000 - val_root_mean_squared_error: 2805.4607
Epoch 3/500
3/3 [==============================] - 0s 66ms/step - loss: 8127967.0000 - root_mean_squared_error: 2850.9590 - val_loss: 7862366.0000 - val_root_mean_squared_error: 2803.9912
Epoch 4/500
3/3 [==============================] - 0s 67ms/step - loss: 8120781.0000 - root_mean_squared_error: 2849.6985 - val_loss: 7853495.0000 - val_root_mean_squared_error: 2802.4087
Epoch 5/500
3/3 [==============================] - 0s 67ms/step - loss: 8112363.5000 - root_mean_squared_error: 2848.2212 - val_loss: 7843888.0000 - val_root_mean_squared_error: 2800.6943
Epoch 6/500
3/3 [==============================] - 0s 67m

In [21]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

Reusing TensorBoard on port 6006 (pid 9016), started 1:32:58 ago. (Use '!kill 9016' to kill it.)

In [22]:
mse_score, rmse_score = model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 89ms/step - loss: 140879.1875 - root_mean_squared_error: 375.3388


In [23]:
X_new = X_train[:3]

In [24]:
X_train[:3]

,Tool Dia. (mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
63,3.0,48.3988,48.3622,0.0366,9.0,7.071112,0.328,0.112
31,2.0,49.1654,49.1434,0.0220,9.0,6.014213,0.089,0.234
71,3.0,48.1323,48.1152,0.0171,18.0,6.727256,0.156,0.179


In [25]:
model.predict(X_new)

1/1 [==============================] - 0s 228ms/step


array([[3964.8083  ,   18.784481],
       [3479.8333  ,   12.940952],
       [4441.0005  ,   21.04578 ]], dtype=float32)

In [26]:
y_train[:3]

,Spindle Speed,Feed (mm/min)
63,3750.0,10.0
31,3250.0,15.0
71,4250.0,20.0


### Model Tuning

In [27]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=100, max_value=500)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
    sampling="log")
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model = keras.Sequential()
    model.add(norm_layer)
    for _ in range(n_hidden):
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(keras.layers.Dense(2))
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    return model

In [30]:
tuner = kt.RandomSearch(
 build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), max_trials=10, overwrite=True,
 directory="Model_Tuning", project_name="my_rnd_search", seed=42)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

tuner.search(X_train, y_train, epochs=150, validation_data=(X_val, y_val), callbacks=[early_stop])

Trial 10 Complete [00h 01m 08s]
val_root_mean_squared_error: 136.66893005371094

Best val_root_mean_squared_error So Far: 106.05316925048828
Total elapsed time: 00h 06m 55s


In [31]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [32]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

In [33]:
best_trial.summary()

Trial 03 summary
Hyperparameters:
n_hidden: 2
n_neurons: 492
learning_rate: 0.001715074355925934
Score: 106.05316925048828


In [34]:
Final_model = tuner.get_best_models(num_models=1)[0]

In [37]:
Final_model.fit(X_train_full, y_train_full, epochs=200, callbacks=[tensorboard_cb])

Epoch 1/200
3/3 [==============================] - 0s 22ms/step - loss: 16507.0332 - root_mean_squared_error: 128.4797
Epoch 2/200
3/3 [==============================] - 0s 25ms/step - loss: 51108.3438 - root_mean_squared_error: 226.0715
Epoch 3/200
3/3 [==============================] - 0s 21ms/step - loss: 21720.7715 - root_mean_squared_error: 147.3797
Epoch 4/200
3/3 [==============================] - 0s 24ms/step - loss: 44638.3906 - root_mean_squared_error: 211.2780
Epoch 5/200
3/3 [==============================] - 0s 25ms/step - loss: 14474.0986 - root_mean_squared_error: 120.3083
Epoch 6/200
3/3 [==============================] - 0s 22ms/step - loss: 57489.7891 - root_mean_squared_error: 239.7703
Epoch 7/200
3/3 [==============================] - 0s 24ms/step - loss: 39696.6836 - root_mean_squared_error: 199.2402
Epoch 8/200
3/3 [==============================] - 0s 18ms/step - loss: 20062.6113 - root_mean_squared_error: 141.6425
Epoch 9/200
3/3 [==============================]

In [38]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9016), started 1:57:14 ago. (Use '!kill 9016' to kill it.)